# All PyZX Features

## Contents:
* [Loading and saving circuits](#circuits)
* [Interacting with Quantomatic](#quantomatic)
* [Optimizing ZX-diagrams](#optimization-zx)
* [Extracting and optimizing circuits](#optimization-circuits)
* [Phase Teleportation](#phase-teleportation)

In [1]:
import sys; sys.path.insert(0,'..')
import random, math, os
import pyzx as zx
from fractions import Fraction

<a id="circuits"></a>
# Loading and saving circuits
The most straightforward way to load a circuit is to use ``zx.Circuit.load`` which tries to figure out in which file format the circuit is given. The supported file formats are QASM, QC and the Quipper ASCII format. The corresponding loading functions are:
* ``zx.Circuit.from_qasm_file``
* ``zx.Circuit.from_quipper_file``
* ``zx.Circuit.from_qc_file``

In [2]:
fname = os.path.join('..','circuits','benchmarking_circuits', 'Fast', 'before', 'mod5_4')
circ = zx.Circuit.load(fname)
# Alternatively we could have done:
# circ = zx.Circuit.from_quipper_file(fname)
circ.gates

As you can see, a circuit is essentially a list of gates (objects of type zx.circuit.Gate). We can draw the circuit using zx.draw():

In [3]:
zx.draw_matplotlib(circ, figsize=(10,2), h_edge_draw='box')

The CCZ gates are drawn in [ZH](https://arxiv.org/abs/1805.02175) notation by default. We can get a pure ZX-diagram if we first convert the circuit to basic (i.e. Clifford+T) gates:

In [4]:
zx.draw_matplotlib(circ.to_basic_gates(), figsize=(10,2), h_edge_draw='box')

Here we've drawn the circuit using the Matplotlib backend, but we can also use the D3 Javascript library to generate a more interactive representation:

In [5]:
zx.draw(circ)

We can ask for simple statistics for this circuit:

In [6]:
print(circ.stats())

Note that a lot of the circuit is hidden in the mysterious "4 gates of a different type" (which are the CCZ gates). We can write the circuit in terms of basic gates to get more accurate Clifford+T statistics:

In [7]:
print(circ.to_basic_gates().stats())

A Circuit can be exported into the supported file formats using
    * ``circ.to_qasm``
    * ``circ.to_quipper``
    * ``circ.to_qc``

In [8]:
print(circ.to_qasm())

In [9]:
print(circ.to_quipper())

In [10]:
print(circ.to_qc())

<a id="quantomatic"></a>
# Interacting with Quantomatic
PyZX allows easy integration with quantomatic.

First of all, Quantomatic graph files can be imported into PyZX:

In [11]:
fname = os.path.join('..','circuits','2-qubit-unitary.qgraph')
with open(fname, 'r') as f:
    g = zx.Graph.from_json(f.read())
zx.draw(g, labels=True)

PyZX saves the names of the vertices:

In [12]:
print(g.vdata(12,'name'))
print(g.vdata(1,'name'))

Because this graph was originally exported from PyZX, it has automatically remembered what its inputs and outputs are:

In [13]:
g.inputs(), g.outputs()

For a graph that originated from Quantomatic we need to tell it what its inputs and outputs are.

This can be done either manually:

    g.set_inputs(tuple_of_inputs); g.set_outputs(tuple_of_outputs)
    
or by PyZX through calling ``g.auto_detect_io()``. This function makes all boundaries pointing to the right inputs, and those pointing to the left inputs. For example:

In [14]:
g.set_inputs(())
g.set_outputs(())
g.auto_detect_io()
print(g.inputs(), g.outputs())

We can also call Quantomatic from PyZX. To do this we first need to tell PyZX where the Quantomatic executable can be found:

In [ ]:
zx.quantomatic.quantomatic_location = os.path.join('path', 'to', 'Quantomatic.jar')

Now, we can load a PyZX graph into Quantomatic using the following line:

In [ ]:
result = zx.quantomatic.edit_graph(g)

This starts Quantomatic with the graph ``g`` loaded. When you are done editing the graph, you simply save the file in Quantomatic, and close it. The result is then loaded and returned.

NOTE1: The Notebook will be blocked until the Quantomatic executable is closed.

NOTE2: Currently this only works with a recent build of Quantomatic that is as of yet only available via the repository, so make sure you are working with an up-to-date branch of Quantomatic.

<a id="optimization-zx"></a>
# Optimizing ZX-diagrams
PyZX contains many functions for optimizing circuits and ZX-diagrams. In this section we will show how these methods work and can be called.

First, let us load a small circuit:

In [15]:
fname = os.path.join('..','circuits','benchmarking_circuits', 'Fast', 'before', 'mod5_4')
circ = zx.Circuit.load(fname).to_basic_gates()
print("original T-count:", zx.tcount(circ))
zx.draw(circ)

The most basic simplification routine for ZX-graphs is ``interior_clifford_simp``. This uses the simplification rules based on spider-fusion, identity removal, pivoting and local complementation until they cannot be applied anymore.

In [16]:
g = circ.to_graph() # We first have to convert the circuit into a ZX-graph
zx.simplify.interior_clifford_simp(g,quiet=False) # if it is not quiet then the amount of reductions in each step is printed
# The following function makes the representation of the graph more compact. 
# It only moves vertices around, and may introduce some identities
# It is recommended to call this function before trying to draw a graph, as otherwise the graph might not be very readable.
g.normalize() 
zx.draw(g)
print("Optimized T-count:", zx.tcount(g))

As you can see this routine has decreased the T-count by a bit, but not by a lot.


Part of the reason is that in this circuit there are still a few phaseless nodes 'trapped' in the middle of the circuit. These nodes are however connected to the boundary, so if we unfuse those boundary nodes we can get rid of the interior nodes. This can be done by calling ``zx.simplify.pivot_boundary_simp``. As this function changes the graph, we call ``interior_clifford_simp`` afterwards to do whatever new simplifications have become available.

In [17]:
zx.simplify.pivot_boundary_simp(g, quiet=False)
zx.draw(g)
zx.simplify.interior_clifford_simp(g,quiet=False)
g.normalize()
zx.draw(g)
print("Optimized T-count:", zx.tcount(g))

Though the graph is smaller now, it hasn't decreased the T-count any further in this case.

The method ``clifford_simp`` is a convenience function that calls ``interior_clifford_simp`` and ``pivot_boundary_simp`` until no further simplifications are found. It is recommended that you call ``clifford_simp`` instead of the other functions.
In order to further reduce the T-count we have to start using more advanced techniques. Namely the process of *gadgetization*:

In [18]:
zx.simplify.pivot_gadget_simp(g, quiet=False)
zx.draw(g)
print("T-count:", zx.tcount(g))

This method combines all T-like phases with a phaseless spider to turn it into a *phase gadget*. In the previous diagram those are the spider pairs floating above the rest of the diagram, which we refer to as its *skeleton*. As you can see, the T-count has not actually been reduced by doing just these pivots, but now the graph has a completely different structure then before, so lets see what happens if we apply ``clifford_simp`` again:

In [19]:
zx.simplify.clifford_simp(g, quiet=False)
g.normalize()
zx.draw(g)
print("Optimized T-count:", zx.tcount(g))

That has reduced the T-count from 22 down to 18!

But we are in fact not done yet. If you look closely at the graph above you will see that a few of the gadgets have exactly the same set of neighbors. Whenever this happens, these phase gadgets can be fused into a single phase gadget by adding the phases together. This procedure is done by ``gadget_simp``:

In [20]:
zx.simplify.gadget_simp(g, quiet=False)
g.normalize()
zx.draw(g)
print("Optimized T-count:", zx.tcount(g))

The T-count has now more than halved! Note that the previous best-known T-count for this circuit was 16, so this is a big improvement.

There is not much we can do now. We do one final round of ``clifford_simp`` to make the graph a bit smaller.

In [21]:
zx.simplify.clifford_simp(g, quiet=False)
g.normalize()
zx.draw(g)

Since it is quite a bit of effort to do all these steps manually, they have been combined into the function ``full_reduce``. This procedure does the following steps:

 1. Run ``clifford_simp``.
 2. Gadgetize the diagram using ``pivot_gadget_simp``.
 4. Run ``clifford_simp``.
 5. Run ``gadget_simp``. If it finds simplifications go back to step 1, otherwise halt.
 
To demonstrate:

In [22]:
g = circ.to_basic_gates().to_graph()
zx.simplify.full_reduce(g,quiet=False)
g.normalize()
zx.draw(g)

The next step is to turn this graph back into a circuit

<a id="optimization-circuits"></a>
# Extracting and optimizing circuits

For extracting circuits out of ZX-graphs there is a method in PyZX called: ``zx.extract_circuit``. This method should always work *when dealing with graphs produced by ``full_reduce``.* There is no guarantee that it can extract circuits from arbitrary ZX-diagrams.

Let's see what this method does when applied to the circuit from the previous section:

In [23]:
fname = os.path.join('..','circuits','benchmarking_circuits', 'Fast', 'before', 'mod5_4')
circ = zx.Circuit.load(fname).to_basic_gates()
print("The original circuit:")
zx.draw(circ)

g = circ.to_graph()
zx.simplify.full_reduce(g,quiet=True)
g.normalize()
print("The optimized ZX-diagram:")
zx.draw(g)

new_circ = zx.extract_circuit(g)
print("The extracted circuit:")
zx.draw(new_circ)

Explaining how ``extract_circuit`` works is out of scope for this notebook for now. If you want to know more you can check out the paper [Graph-theoretic Simplification of Quantum Circuits with the ZX-calculus](https://arxiv.org/abs/1902.03178) or the more recent (but more involved) [There and back again: A circuit extraction tale](https://arxiv.org/abs/2003.01664).

As you can see, the extracted circuit looks quite different from the original circuit, so how can we be sure that they actually represent the same unitary? PyZX allows you to convert ZX-diagrams into the tensors they represent using numpy. In this way we can directly compare the unitaries and see that they are equal:

In [24]:
# This method checks whether the two given ZX-graphs or circuits have the same tensor representation up to some nonzero scalar
zx.compare_tensors(circ, new_circ)

This method uses ``zx.tensorfy`` under the hood to turn the circuits into tensors. This method is not optimized for memory usage, so it will run out of memory quite quickly (sometimes even for circuits with 9 or 10 qubits). It is only usable for testing small circuits and ZX-diagrams.

For larger circuits PyZX also offers a different method to check whether two circuits are equal:

In [25]:
circ.verify_equality(new_circ)

This method takes the composition of the first circuit with the adjoint of the second and simplifies the resulting circuit with ``full_reduce``. If it succeeds, it returns ``True``, if it does not it returns ``False``. This means that if this method succeeds, that is very likely that the 2 circuits are equal (it would be very unlikely that some bug in the rewriting engine would cancel out in exactly the right way to be able to reduce the circuit to the identity). However, if it returns ``False`` it might simply mean that the rewrite engine is not powerful enough to verify the equality of the two circuits.

``extract_circuit`` often produces circuits that are clearly not optimal as can be seen in the series of Hadamard gates at the end of the above extracted circuit. PyZX offers a circuit optimization method that takes care of this obvious kind of suboptimality:

In [26]:
print(new_circ.to_basic_gates().stats())
optimized_circ = zx.optimize.basic_optimization(new_circ.to_basic_gates(),do_swaps=False).to_basic_gates()
print(optimized_circ.stats())
zx.draw(optimized_circ)

``basic_optimization`` commutes gates past Hadamards in order to find matching CNOT, CZ and Hadamard gates that can be cancelled. Depending on the circuit it can find significant reductions in the amount of Hadamard gates, which is useful for the next optimization routine. Note that we gave it the argument `do_swaps=False`. When this is instead set to True, it employs more powerful optimization techniques involving transforming adjacent CNOT gates into SWAPs. While for larger circuits this often leads to better results, for the above circuit it actually increases the 2-qubit gate-count (from 25 to 27).

As you can see, the optimized circuit only contains 2 Hadamard gates, and in fact the leftmost Hadamard gate could be commuted past the CNOTs and CZs to its left so that the entire interior of the circuit is free of Hadamards. Such a Hadamard-free circuit is called a *phase polynomial* circuit, and there are specific techniques for optimizing these types of circuits. PyZX offers a method that finds phase polynomial sub-circuits and runs an optimization routine on them:

In [27]:
final_circ = zx.optimize.phase_block_optimize(optimized_circ).to_basic_gates()
final_circ = zx.optimize.basic_optimization(final_circ) # We call this again, as it does some extra processing
print(final_circ.stats())
zx.draw(final_circ)

Let's check once more that this circuit is still equal to the original circuit:

In [28]:
zx.compare_tensors(circ, final_circ)

And finally lets output this circuit in a format that is usable in other software:

In [29]:
print(final_circ.to_qasm())

<a id="phase-teleportation"></a>
# Phase teleportation

The above procedure of simplifying a diagram, and extracting a new circuit directly from it worked quite well in the example given above, but unfortunately in other cases it can actually *increase* the total gate-count (although it never increases the amount of T-gates).

To get around this issue PyZX offers the *phase teleportation* routine. This uses the diagrammatic simplification only as information to inform when phase gates can be combined in the original circuit, but otherwise it leaves the circuit intact. The details can be found in [Reducing T-count with the ZX-calculus](https://arxiv.org/abs/1903.10477).

In [30]:
fname = os.path.join('..','circuits','benchmarking_circuits', 'Fast', 'before', 'mod5_4')
circ = zx.Circuit.load(fname).to_basic_gates()
print("The original circuit:")
zx.draw(circ)
g = circ.to_graph()
zx.simplify.teleport_reduce(g)
print("Circuit after phase teleportation:")
zx.draw(g)

As you can see the circuit is exactly the same, except that it has less phase gates. Let's verify that it is indeed still the same circuit:

In [31]:
zx.compare_tensors(circ, g)

<a id="phase-teleportation"></a>
# Two-qubit gate count optimisation

To illustrate how the previous strategy can in fact worsen the optimised circuit in terms of total gate count and two-qubit gate count, let's look at another, bigger, circuit:

In [32]:
fname = os.path.join('..','circuits','benchmarking_circuits', 'Fast', 'before', 'csum_mux_9_corrected')
circ = zx.Circuit.load(fname).to_basic_gates()
print("The original circuit:")
print(circ.stats(), end='\n\n')

g = circ.to_graph()
zx.simplify.full_reduce(g,quiet=True)
g.normalize()
new_circ = zx.extract_circuit(g)
optimized_circ = zx.optimize.basic_optimization(new_circ.to_basic_gates(),do_swaps=False).to_basic_gates()
print("The optimised circuit:")
print(optimized_circ.stats())

The original circuit:
Circuit csum_mux_9_corrected on 30 qubits with 448 gates.
        196 is the T-count
        252 Cliffords among which
        168 2-qubit gates (168 CNOT, 0 other) and
        56 Hadamard gates.

The optimised circuit:
Circuit  on 30 qubits with 496 gates.
        84 is the T-count
        412 Cliffords among which
        349 2-qubit gates (314 CNOT, 35 other) and
        28 Hadamard gates.


As you can see the both the total gate count and two-qubit gate counts have in fact *increased* from the original. The T-count has been significantly reduced, but we could have achieved this by using phase teleportation. Let's see if using ``phase_block_optimize`` can help us here:

In [33]:
final_circ = zx.optimize.phase_block_optimize(optimized_circ).to_basic_gates()
final_circ = zx.optimize.basic_optimization(final_circ) # We call this again, as it does some extra processing
print(final_circ.stats())

Circuit  on 30 qubits with 539 gates.
        74 is the T-count
        465 Cliffords among which
        391 2-qubit gates (319 CNOT, 72 other) and
        28 Hadamard gates.


As demonstrated here, ``phase_block_optimize`` always attempts to reduce T-count, but at the cost of many additional two-qubit gates. 

Let's look at a different strategy we can use, specifically with the goal of minimising the number of two-qubit gates (while maintaining the same T-count reductions).

The first step is to apply phase teleportation to our circuit. This allows us to reap the rewards of ``full_reduce`` without the downside of increased gates after extraction.

In [34]:
fname = os.path.join('..','circuits','benchmarking_circuits', 'Fast', 'before', 'csum_mux_9_corrected')
circ = zx.Circuit.load(fname).to_basic_gates()
g = circ.to_graph()
zx.simplify.teleport_reduce(g)
print(zx.Circuit.from_graph(g).stats())

Circuit  on 30 qubits with 364 gates.
        84 is the T-count
        280 Cliffords among which
        168 2-qubit gates (168 CNOT, 0 other) and
        56 Hadamard gates.


We can then apply a method which *selectively* applies only the transformations of ``full_reduce`` which reduce the two-qubit gate count of the resulting circuit. To do this we must first transform the diagram into what is called graph-like form. This allows us to evaluate graph theoretic properties of the diagram. In particular if we assert that all transformations preserve a property called *cflow*, then instead of the usual extraction function we can use a method called ``simple_extract``. 

The method which applies the selective simplification of our ZX-diagram is called ``flow_2Q_simp``. The full details can be found in [Causal flow preserving optimisation of quantum circuits in the ZX-calculus](https://arxiv.org/abs/2312.02793).

Let's apply it and see how it does:

In [35]:
zx.to_graph_like(g)
zx.flow_2Q_simp(g)
g.normalize()
new_circ = zx.extract_simple(g)
optimized_circ = zx.optimize.basic_optimization(new_circ.to_basic_gates(),do_swaps=False).to_basic_gates()
print(optimized_circ.stats())

Circuit  on 30 qubits with 288 gates.
        84 is the T-count
        204 Cliffords among which
        140 2-qubit gates (125 CNOT, 15 other) and
        28 Hadamard gates.


As you can see this significantly reduces the number of gates. This method will *never* increase the number of two-qubit gates. Let us finally confirm its equality:

In [36]:
circ.verify_equality(optimized_circ)

True